In [ ]:
import numpy as np

def read_fasta_file(filename):
    """
    Reads the given FASTA file f and returns a dictionary of sequences.

    Lines starting with ';' in the FASTA file are ignored.
    """
    sequences_lines = {}
    current_sequence_lines = None
    with open(filename) as fp:
        for line in fp:
            line = line.strip()
            if line.startswith(';') or not line:
                continue
            if line.startswith('>'):
                sequence_name = line.lstrip('>')
                current_sequence_lines = []
                sequences_lines[sequence_name] = current_sequence_lines
            else:
                if current_sequence_lines is not None:
                    current_sequence_lines.append(line)
    sequences = {}
    for name, lines in sequences_lines.items():
        sequences[name] = ''.join(lines)
    return sequences

In [ ]:
a=read_fasta_file("true-ann1.fa")

def map_truestate_to_3statemarkov(strn):
    out=[]
    for i in strn:
        if i == 'C':
            out.append(0)
        elif i == 'N':
            out.append(1)
        elif i == 'R':
            out.append(2)
        else:
            raise Exception("Input should be a either N, C or R")
    return out

def map_truestate_to_7statemarkov(strn):
    out=[None]*len(strn)
    for i in range(len(strn)):
        if strn[i] == 'C':
            if out[i-1]==0:
                out[i]=(1)
            elif out[i-1]==1:
                out[i]=(2)
            else:
                out[i]=(0)
        elif strn[i] == 'N':
            out[i]=(3)
        elif strn[i] == 'R':
            if out[i-1]==4:
                out[i]=(5)
            elif out[i-1]==5:
                out[i]=(6)
            else:
                out[i]=(4)
        else:
            raise Exception("Input should be a either N, C or R")
    return out


In [ ]:
def translate_observations_to_indices(obs):
    mapping = {'a': 0, 'c': 1, 'g': 2, 't': 3}
    return [mapping[symbol.lower()] for symbol in obs]

def translate_indices_to_observations(indices):
    mapping = ['a', 'c', 'g', 't']
    return ''.join(mapping[idx] for idx in indices)

class hmm:
    def __init__(self, init_probs, trans_probs, emission_probs):
        self.init_probs = init_probs
        self.trans_probs = trans_probs
        self.emission_probs = emission_probs



init_probs_3_state = np.array(
    [0.00, 1.00, 0.00]
)

trans_probs_3_state = np.array([
    [0.90, 0.10, 0.00],
    [0.05, 0.90, 0.05],
    [0.00, 0.10, 0.90],
])

emission_probs_3_state = np.array([
    #   A     C     G     T
    [0.40, 0.15, 0.20, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.20, 0.40, 0.30, 0.10],
])

hmm_3_state = hmm(init_probs_3_state,
                  trans_probs_3_state,
                  emission_probs_3_state)

init_probs_7_state = np.array(
    [0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00]
)



trans_probs_7_state = np.array([
    [0.90, 0.00, 0.00, 0.10, 0.00, 0.00, 0.00],
    [0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00],
    [0.05, 0.00, 0.00, 0.90, 0.05, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.10, 0.90, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00],    
])

emission_probs_7_state = np.array([
    #   A     C     G     T
    [0.30, 0.25, 0.25, 0.20],
    [0.20, 0.35, 0.15, 0.30],
    [0.40, 0.15, 0.20, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.20, 0.40, 0.30, 0.10],
    [0.30, 0.20, 0.30, 0.20],
    [0.15, 0.30, 0.20, 0.35],
])

hmm_7_state = hmm(init_probs_7_state,
                  trans_probs_7_state,
                  emission_probs_7_state)


In [ ]:
import math
def log(x):
    if x == 0:
        return float("-inf")
    else:
        return math.log(x)

#Fill out the blanks to get the Viterbi algorithm:

def viterbi(obs, hmm):
    X = translate_observations_to_indices(obs)
    N = len(X)
    K = len(hmm.init_probs)
    V = np.zeros((K,N))

    init_probs=np.log(hmm.init_probs)
    trans_probs=np.log(hmm.trans_probs)
    emission_probs=np.log(hmm.emission_probs)

    for i in range(K):
        V[i][0]=init_probs[i]
    for i in range(1, N):
        if i%100000==0:
            print(i) 
        for n in range(K):
            E = emission_probs[n][X[i]]
            T = trans_probs[:,n]+V[:,i-1]
            V[n][i]= E + np.max(T)
    return V


def backtrack(V, hmm):
    assert len(V) == len(hmm.init_probs)

    init_probs = np.log(hmm.init_probs)
    trans_probs = np.log(hmm.trans_probs)
    emission_probs = np.log(hmm.emission_probs)

    N = len(V[0])
    i = N - 1
    k = np.argmax(V[:,N-1])
    o = []

    while i >= 0:
        o.append(k)
        k = np.argmax(V[:,i-1] + trans_probs[:,k] )
        i-=1
        if i % 100000 == 0: print(i)
    return o[::-1]




In [ ]:
def compute_accuracy(true_ann, pred_ann):
    if len(true_ann) != len(pred_ann):
        return 0.0
    return sum(1 if true_ann[i] == pred_ann[i] else 0 
               for i in range(len(true_ann))) / len(true_ann)

true_ann = map_truestate_to_7statemarkov(read_fasta_file("true-ann1.fa")["true-ann1"])

obs = read_fasta_file("genome1.fa")
pred_ann = backtrack(viterbi(obs["genome1"], hmm_7_state),hmm_7_state)

compute_accuracy(true_ann, pred_ann)

#question 1 = 0.2546272728794061

In [ ]:
true_ann = map_truestate_to_7statemarkov(read_fasta_file("true-ann2.fa")["true-ann2"])

obs = read_fasta_file("genome2.fa")
pred_ann = backtrack(viterbi(obs["genome2"], hmm_7_state),hmm_7_state)

compute_accuracy(true_ann, pred_ann)

#question 2 = 0.2181276382159499

In [ ]:
true_ann = map_truestate_to_3statemarkov(read_fasta_file("true-ann1.fa")["true-ann1"])

obs = read_fasta_file("genome1.fa")
pred_ann = backtrack(viterbi(obs["genome1"], hmm_3_state),hmm_3_state)

compute_accuracy(true_ann, pred_ann)

#question 3 = 0.31873349812490653

In [ ]:
true_ann = map_truestate_to_3statemarkov(read_fasta_file("true-ann2.fa")["true-ann2"])

obs = read_fasta_file("genome2.fa")
pred_ann = backtrack(viterbi(obs["genome2"], hmm_3_state),hmm_3_state)

compute_accuracy(true_ann, pred_ann)

#question 4 = 0.35088368223162264

Attempt at training

In [ ]:
#We train the init_probs by setting them to their true initial state, this is fairly straight-forward as we know the init-state from the true-ann1.fa file

true_str = read_fasta_file("true-ann1.fa")["true-ann1"]

def train_init_state(true_str, states):
    if states == 3:
        init_state=map_truestate_to_3statemarkov(true_str[0])
        I = np.array(
            [0.00, 1.00, 0.00]
            )
    elif states == 7:
        init_state=map_truestate_to_7statemarkov(true_str[0])
        I = np.array(
            [0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00]
            )

    return I


In [ ]:
#to train the trans_probs we need to make a count of all the transitions.

true_str = read_fasta_file("true-ann1.fa")["true-ann1"]

def train_trans_probs(true_str, states):
    if states == 3:
        true_str = map_truestate_to_3statemarkov(true_str)
    elif states == 7:
        true_str = map_truestate_to_7statemarkov(true_str)
    else:
        raise Exception("State number not supported")
    
    T = np.zeros((states,states))
    for i in range(1,len(true_str)):
        T[true_str[i-1]][true_str[i]]+=1
    for i in range(states):
        counts = 0
        for n in range(states):
            counts+=T[i][n]
        for n in range(states):
            T[i][n]=T[i][n]/counts

    return(T)



In [ ]:
true_str = read_fasta_file("true-ann1.fa")["true-ann1"]
nuc_str = read_fasta_file("genome1.fa")["genome1"]

def train_emission_probs(nuc_str,true_str, states):
    if states == 3:
        true_lst = map_truestate_to_3statemarkov(true_str)
    elif states == 7:
        true_lst = map_truestate_to_7statemarkov(true_str)
    else:
        raise Exception("State number not supported")
    
    nuc_str = translate_observations_to_indices(nuc_str)

    T = np.zeros((states,4))

    for i in range(1,len(true_lst)):
        T[true_lst[i]][nuc_str[i]]+=1
    for i in range(states):
        counts = 0
        for n in range(4):
            counts+=T[i][n]
        for n in range(4):
            T[i][n]=T[i][n]/counts

    return(T)


In [ ]:
#and put it all together

def trained_hmm(true_str, nuc_str, states):
    out=hmm(train_init_state(true_str, states),
    train_trans_probs(true_str,states),
    train_emission_probs(nuc_str,true_str, states))
    return out




In [ ]:

true_str1 = read_fasta_file("true-ann1.fa")["true-ann1"]
nuc_str1 = read_fasta_file("genome1.fa")["genome1"]

hmm_3_state1=trained_hmm(true_str1,nuc_str1,3)

hmm_7_state1=trained_hmm(true_str1,nuc_str1,7)


true_str2 = read_fasta_file("true-ann2.fa")["true-ann2"]
nuc_str2 = read_fasta_file("genome2.fa")["genome2"]

hmm_3_state2=trained_hmm(true_str2,nuc_str2,3)

hmm_7_state2=trained_hmm(true_str2,nuc_str2,7)


In [ ]:
true_ann = map_truestate_to_7statemarkov(read_fasta_file("true-ann2.fa")["true-ann2"])

obs = read_fasta_file("genome2.fa")
pred_ann = backtrack(viterbi(obs["genome2"], hmm_7_state1),hmm_7_state1)

print(len(true_ann),len(pred_ann))

compute_accuracy(true_ann, pred_ann)

#question 5 = 0.7800618136681913

In [ ]:
true_ann = map_truestate_to_3statemarkov(read_fasta_file("true-ann2.fa")["true-ann2"])

obs = read_fasta_file("genome2.fa")
pred_ann = backtrack(viterbi(obs["genome2"], hmm_3_state1),hmm_3_state1)

print(len(true_ann),len(pred_ann))

compute_accuracy(true_ann, pred_ann)

#question 6 = 0.57368917266

In [ ]:
true_ann = map_truestate_to_7statemarkov(read_fasta_file("true-ann1.fa")["true-ann1"])

obs = read_fasta_file("genome1.fa")
pred_ann = backtrack(viterbi(obs["genome1"], hmm_7_state2),hmm_7_state2)

print(len(true_ann),len(pred_ann))

compute_accuracy(true_ann, pred_ann)

#question 7 = 0.7608863116288184

In [ ]:
true_ann = map_truestate_to_3statemarkov(read_fasta_file("true-ann1.fa")["true-ann1"])

obs = read_fasta_file("genome1.fa")
pred_ann = backtrack(viterbi(obs["genome1"], hmm_3_state2),hmm_3_state2)

print(len(true_ann),len(pred_ann))

compute_accuracy(true_ann, pred_ann)

#question 8 = 0.5909796857227841